In [1]:
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding
from keras.preprocessing import sequence

Using TensorFlow backend.


In [0]:
import numpy as np
# save np.load
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

# call load_data with allow_pickle implicitly set to true
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=20000)

# restore np.load for future normal usage
np.load = np_load_old

In [0]:
# 훈련셋과 검증셋 분리
x_val = x_train[20000:]
y_val = y_train[20000:]
x_train = x_train[:20000]
y_train = y_train[:20000]

In [0]:
# 데이터셋 전처리
x_train = sequence.pad_sequences(x_train, maxlen=200)
x_val = sequence.pad_sequences(x_val, maxlen=200)
x_test = sequence.pad_sequences(x_test, maxlen=200)
# 각 훈련 데이터는 문장의 길이가 다르기 때문에, 모델이 처리할 수 있도록 길이를 동일하게 해주어야 한다.
# 이때 사용하는 것이 pad_sequences()이다.
# 길이는 maxlen에 넣는 값으로 정해진다.
# 훈련 데이터가 정한 길이를 초과하면 초과분을 삭제하고, 부족하면 0으로 채운다.

In [5]:
# 모델 구성
model = Sequential()
model.add(Embedding(20000, 128))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))
# 출력층은 활성화 함수로 시그모이드 함수를 갖는 뉴런 하나를 사용한다.
# 시그모이드 함수를 사용하면 0 또는 1의 결과를 도출해낼 수 있기 때문에 분류해야하는 카테고리가 두 개일 경우에 출력층에서 사용한다.

W0804 07:00:12.860016 139697656129408 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0804 07:00:12.880299 139697656129408 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0804 07:00:12.882573 139697656129408 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [6]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 128)         2560000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 2,691,713
Trainable params: 2,691,713
Non-trainable params: 0
_________________________________________________________________
None


In [7]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) # 모델 학습과정 설정

hist = model.fit(x_train, y_train, epochs=25, batch_size=64, validation_data=(x_val, y_val))
# 모델 학습시키기

W0804 07:00:13.124430 139697656129408 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0804 07:00:13.150784 139697656129408 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3376: The name tf.log is deprecated. Please use tf.math.log instead.

W0804 07:00:13.156845 139697656129408 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0804 07:00:13.982390 139697656129408 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. P

Train on 20000 samples, validate on 5000 samples
Epoch 1/25
20000/20000 [==============================] - 82s 4ms/step - loss: 0.4438 - acc: 0.7892 - val_loss: 0.3747 - val_acc: 0.8382
Epoch 2/25
20000/20000 [==============================] - 81s 4ms/step - loss: 0.2346 - acc: 0.9085 - val_loss: 0.3236 - val_acc: 0.8616
Epoch 3/25
20000/20000 [==============================] - 80s 4ms/step - loss: 0.1395 - acc: 0.9491 - val_loss: 0.3973 - val_acc: 0.8702
Epoch 4/25
20000/20000 [==============================] - 80s 4ms/step - loss: 0.1133 - acc: 0.9598 - val_loss: 0.4681 - val_acc: 0.8538
Epoch 5/25
20000/20000 [==============================] - 80s 4ms/step - loss: 0.0599 - acc: 0.9799 - val_loss: 0.5400 - val_acc: 0.8468
Epoch 6/25
20000/20000 [==============================] - 79s 4ms/step - loss: 0.0600 - acc: 0.9800 - val_loss: 0.6000 - val_acc: 0.8022
Epoch 7/25
20000/20000 [==============================] - 79s 4ms/step - loss: 0.0748 - acc: 0.9736 - val_loss: 0.5386 - val_acc:

In [10]:
scores = model.evaluate(x_train, y_train, verbose=0) # 테스트 데이터에 대해서 정확도 평가
print("정확도: %.8f%%" % (scores[1]*100))

정확도: 100.00000000%


In [9]:
# 모델 평가
loss_and_metrics = model.evaluate(x_test, y_test, batch_size=64)
print('## evaluation loss and_metrics ##')
print(loss_and_metrics)

25000/25000 [==============================] - 18s 733us/step
## evaluation loss and_metrics ##
[1.0024222064208985, 0.8406]
